# Artificial Neural Network template

## ======= Import the data set ========

In [1]:
import pandas as pd
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

## ======= Categorical Data =========

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
print ('Gender:', set(x[:,2]), '\nCountry:', set(x[:,1]))
label_encoder = LabelEncoder()
x[:, 2] = label_encoder.fit_transform(x[:, 2]) #Female:0, Male:1
x[:, 1] = label_encoder.fit_transform(x[:, 1]) 
one_hot_encoder = OneHotEncoder(categorical_features = [1])
x = one_hot_encoder.fit_transform(x).toarray() #France:100, Spain:001, Germany:010
x = x[:, 1:]

Gender: {'Female', 'Male'} 
Country: {'Germany', 'Spain', 'France'}


/Users/LianshuiZhao/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/LianshuiZhao/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


## ======= Splitting the dataset ========

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

## ========= Feature Scaling ========

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## ======= Construct the ANN ========

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def build_classifier(_optimizer):
    ### ======== Initializing the ANN ======
    classifier = Sequential()
    
    ### ======== add the input layer and the hidden layers and output layer =====
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim = x_train.shape[1])) #1st
    #classifier.add(Dropout(p=0.1)) #dealwith overfitting
    
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu')) #2nd
    #classifier.add(Dropout(p=0.1)) #dealwith overfitting
    
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid')) #output
    
    ### ======= Compiling the ANN ========
    classifier.compile(optimizer=_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

Using TensorFlow backend.


## ======== Applying Grid Search to find the best model and the best parameters ========

In [6]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn=build_classifier)
parameters = [{'batch_size': [10, 20],
              'epochs': [20, 40],
              '_optimizer': ['adam']}] #or try optimizer'rmsprop'
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs=-1) 
grid_search = grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print('The best accuracy:', best_accuracy)
print('The best parameters:', best_parameters)

Epoch 1/40
8000/8000 [==============================] - 1s 122us/step - loss: 0.5324 - acc: 0.7948
Epoch 2/40
8000/8000 [==============================] - 1s 101us/step - loss: 0.4346 - acc: 0.7960
Epoch 3/40
8000/8000 [==============================] - 1s 99us/step - loss: 0.4308 - acc: 0.7960
Epoch 4/40
8000/8000 [==============================] - 1s 100us/step - loss: 0.4284 - acc: 0.7960
Epoch 5/40
8000/8000 [==============================] - 1s 99us/step - loss: 0.4257 - acc: 0.7960
Epoch 6/40
8000/8000 [==============================] - 1s 100us/step - loss: 0.4223 - acc: 0.7964
Epoch 7/40
8000/8000 [==============================] - 1s 100us/step - loss: 0.4190 - acc: 0.8210
Epoch 8/40
8000/8000 [==============================] - 1s 100us/step - loss: 0.4172 - acc: 0.8243
Epoch 9/40
8000/8000 [==============================] - 1s 105us/step - loss: 0.4154 - acc: 0.8271
Epoch 10/40
8000/8000 [==============================] - 1s 100us/step - loss: 0.4140 - acc: 0.8296
Epoch 11/40

## ========= Applying the K-Fold cross validation ======

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
classifier = KerasClassifier(build_fn = build_classifier('adam'), batch_size = 20, epochs = 40)

## ====== Predict and make confusion matrix =======

In [9]:
classifier = build_classifier('adam')
classifier.fit(x_train, y_train, batch_size=20, epochs=40)
y_pred_train = classifier.predict(x_train)
y_pred_train = (y_pred_train > 0.5)
y_pred_test = classifier.predict(x_test)
y_pred_test = (y_pred_test > 0.5)

Epoch 1/40
8000/8000 [==============================] - 1s 165us/step - loss: 0.5171 - acc: 0.7961
Epoch 2/40
8000/8000 [==============================] - 1s 113us/step - loss: 0.4309 - acc: 0.7960
Epoch 3/40
8000/8000 [==============================] - 1s 120us/step - loss: 0.4254 - acc: 0.7960
Epoch 4/40
8000/8000 [==============================] - 1s 104us/step - loss: 0.4204 - acc: 0.8093
Epoch 5/40
8000/8000 [==============================] - 1s 102us/step - loss: 0.4152 - acc: 0.8274
Epoch 6/40
8000/8000 [==============================] - 1s 101us/step - loss: 0.4114 - acc: 0.8309
Epoch 7/40
8000/8000 [==============================] - 1s 104us/step - loss: 0.4085 - acc: 0.8313
Epoch 8/40
8000/8000 [==============================] - 1s 105us/step - loss: 0.4065 - acc: 0.8344
Epoch 9/40
8000/8000 [==============================] - 1s 104us/step - loss: 0.4042 - acc: 0.8345
Epoch 10/40
8000/8000 [==============================] - 1s 103us/step - loss: 0.4031 - acc: 0.8341
Epoch 11/

In [10]:
from sklearn.metrics import confusion_matrix
cm_train = confusion_matrix(y_train, y_pred_train)
cm_test = confusion_matrix(y_test, y_pred_test, labels=[0, 1]) #lables indicate the order of the results
print ('The accuracy for the training set is %f.\n'%((cm_train[0][0]+cm_train[1][1])/(x_train.shape[0])))
print ('The accuracy for the test set is %f.\n'%((cm_test[0][0]+cm_test[1][1])/(sum(sum(cm_test)))))

The accuracy for the training set is 0.836250.

The accuracy for the test set is 0.840500.



## ======== Predicting a single new observation ========

In [12]:
import numpy as np
new_pred = classifier.predict(sc.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print ('Exited?', (new_pred > 0.5)[0, 0])

Exited? False


/Users/LianshuiZhao/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
